In [10]:
import requests
from bs4 import BeautifulSoup
import re
import os

# Configuração
url_base = "https://dados.ciga.sc.gov.br/dataset/domsc-publicacoes-de-05-2025"
site_prefix = "https://dados.ciga.sc.gov.br"
os.makedirs("data", exist_ok=True)

# Pega o HTML da página principal do mês
resp = requests.get(url_base)
soup = BeautifulSoup(resp.text, "html.parser")

# Coleta links para as subpáginas de recurso
padrao_pagina = re.compile(r'^/dataset/.+/resource/[\w-]+$')
paginas_recursos = sorted(
    set(site_prefix + l['href'] for l in soup.find_all("a", href=True) if padrao_pagina.match(l['href'])),
    reverse=True
)

# Função para extrair link .zip com padrão 1600_2359
def encontrar_link_zip(pagina_recurso):
    try:
        resp = requests.get(pagina_recurso)
        soup = BeautifulSoup(resp.text, "html.parser")
        link_zip = soup.find("a", href=True, text=re.compile(r'cigadados_publicacoes_\d{4}_\d{2}_\d{2}_1600_2359\.zip$'))
        if link_zip:
            href = link_zip["href"]
            return href if href.startswith("http") else site_prefix + href
    except Exception as e:
        print(f"[!] Erro ao acessar {pagina_recurso}: {e}")
    return None

# Armazena os links válidos
links_validos = []
for pagina in paginas_recursos:
    link = encontrar_link_zip(pagina)
    if link:
        links_validos.append(link)

# Remove duplicatas
links_validos = list(set(links_validos))

# Baixa todos os arquivos encontrados
for link_final in links_validos:
    print(f"🔗 Baixando: {link_final}")
    nome_arquivo = os.path.basename(link_final)
    destino = os.path.join("data", nome_arquivo)
    
    if os.path.exists(destino):
        print(f"✅ Já existe: {nome_arquivo}")
        continue

    try:
        r = requests.get(link_final)
        with open(destino, "wb") as f:
            f.write(r.content)
        print(f"✅ Salvo em: {destino}")
    except Exception as e:
        print(f"[!] Falha no download de {nome_arquivo}: {e}")

/var/folders/ln/bkz1gphd7nx5j8h_b8_4k4lr0000gn/T/ipykernel_57186/365171068.py:27: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  link_zip = soup.find("a", href=True, text=re.compile(r'cigadados_publicacoes_\d{4}_\d{2}_\d{2}_1600_2359\.zip$'))


🔗 Baixando: https://dados.ciga.sc.gov.br/dataset/6fb88011-fc0b-4d0d-bceb-ab59a3811bce/resource/116dbe75-fbe9-456d-8c32-22e818f48925/download/cigadados_publicacoes_2025_05_05_1600_2359.zip
✅ Salvo em: data/cigadados_publicacoes_2025_05_05_1600_2359.zip
🔗 Baixando: https://dados.ciga.sc.gov.br/dataset/6fb88011-fc0b-4d0d-bceb-ab59a3811bce/resource/59ccd448-b939-490e-a028-16a28bdb84da/download/cigadados_publicacoes_2025_05_03_1600_2359.zip
✅ Salvo em: data/cigadados_publicacoes_2025_05_03_1600_2359.zip
🔗 Baixando: https://dados.ciga.sc.gov.br/dataset/6fb88011-fc0b-4d0d-bceb-ab59a3811bce/resource/ed0c462e-8c00-47f5-8e4f-6fc4ecb07fd6/download/cigadados_publicacoes_2025_05_06_1600_2359.zip
✅ Já existe: cigadados_publicacoes_2025_05_06_1600_2359.zip
🔗 Baixando: https://dados.ciga.sc.gov.br/dataset/6fb88011-fc0b-4d0d-bceb-ab59a3811bce/resource/38eadb5a-0e26-4341-b53e-bbb83ec7b9e0/download/cigadados_publicacoes_2025_05_04_1600_2359.zip
✅ Salvo em: data/cigadados_publicacoes_2025_05_04_1600_2359.

In [11]:
import zipfile
import json

zip_path = "data/cigadados_publicacoes_2025_05_06_1600_2359.zip"

with zipfile.ZipFile(zip_path, "r") as zip_file:
    # Pega o primeiro arquivo .json dentro do zip
    json_filename = [name for name in zip_file.namelist() if name.endswith(".json")][0]
    
    with zip_file.open(json_filename) as f:
        data = json.load(f)

# Agora você pode acessar:
print(data["edicoes_ordinarias_exclusivas"][0].keys())

dict_keys(['url', 'data', 'municipio', 'atos'])
